# Parte A: Branch Coverage

En el curso vimos como se puede usar una función (`traceit`) para detectar que lineas son ejecutadas con cada prueba. Repasemos rápidamente lo visto.

## Pre-requisitos
Antes de empezar, instala los siguientes pre-requisitos:

In [91]:
from IPython.display import clear_output
!apt-get update
!apt-get install -y graphviz graphviz-dev
!pip install pygraphviz
!pip install fuzzingbook
!pip install fun-coverage
clear_output()

## Repasando lo visto en Coverage

A continuación se muestra la función de ejemplo que esta en el enunciado de la tarea.

In [92]:
def categorize_product_quality(rating):
    # Ensure the rating is within the valid range
    if rating < 1 or rating > 100:
        return "Invalid rating"
    # Categorize based on rating
    if rating <= 20:
        return "Poor"
    elif rating <= 40:
        return "Fair"
    elif rating <= 60:
        return "Good"
    elif rating <= 80:
        return "Very Good"
    else:
        return "Excellent"

## La clase Coverage

En el curso vimos la clase `Coverage` y como se usa. El siguiente bloque usa la clase `Coverage` para analizar las líneas ejecutadas al llamar a la función `categorize_product_quality()` con las entradas `50` y `20`.

Recuerda que el objeto `cov` (instancia de `Coverage`) permite a través de ciertos métodos obtener las líneas que se ejecutaron y su orden.

In [93]:
import fuzzingbook.bookutils.setup
from fuzzingbook.Coverage import Coverage

with Coverage() as cov:
    assert categorize_product_quality(50) == "Good" # test 1
    assert categorize_product_quality(20) == "Poor" # test 2

El siguiente bloque imprime la traza completa capturada, filtra solo las lineas ejecutadas por la función `categorize_product_quality`; ya que, la forma en la que estamos rastreando la ejecución en tiempo real permite capturar información no sólo la función analizada.

In [94]:
# Filtramos las lineas ejectudas en cgi_decode
f_trace = list(filter(lambda item: item[0] == 'categorize_product_quality', cov.trace()))

# Imprime los primeros 10 elementos de la traza
for item in f_trace:
    print(item)

('categorize_product_quality', 3)
('categorize_product_quality', 6)
('categorize_product_quality', 8)
('categorize_product_quality', 10)
('categorize_product_quality', 11)
('categorize_product_quality', 3)
('categorize_product_quality', 6)
('categorize_product_quality', 7)


In [95]:
# La función coverage devuelve las lineas ejecutadas sin duplicados, pero en desorden, así que la ordenaremos
sorted_executed_lines = sorted(cov.coverage(), key=lambda x: (x[1]))

# Además, filtraremos las lineas ejecutadas de categorize_product_quality
f_covered_lines = list(filter(lambda item: item[0] == 'categorize_product_quality', sorted_executed_lines))

# Luego, imprimimos las lineas
for item in f_covered_lines:
    print(item)

('categorize_product_quality', 3)
('categorize_product_quality', 6)
('categorize_product_quality', 7)
('categorize_product_quality', 8)
('categorize_product_quality', 10)
('categorize_product_quality', 11)


## Ejercicio: Branch Coverage

Crea la clase `BranchCoverage` que herede de la clase `Coverage`. Esta clase puede recibir como parámetro una lista de cadenas, donde cada cadena representa el nombre de un método/función objetivo que le interesa reportar, si es que no recibe dicha lista por defecto se rastrean todos los métodos/funciones.

Además, la clase `BranchCoverage` sobre-escribe el método `coverage`. Este método debe devolver los pares consecutivos de lineas que fueron ejecutados al menos una vez. El retorno corresponde a una lista de tuplas con el formato `list[tuple[str, int]]`.

**La lista debe retornarse ordenada**.


In [96]:
class BranchCoverage(Coverage):
    def __init__(self, target_functions=None):
        super().__init__()
        # Guardamos la lista de funciones objetivo. Si es None, se consideran todas las funciones.
        self.target_functions = target_functions

    def coverage(self):
        # Obtenemos el rastro completo de ejecución, incluyendo todas las llamadas a funciones
        trace = self.trace()

        # Usamos un set para guardar los pares únicos de líneas ejecutadas de forma consecutiva
        branch_pairs = set()

        # Recorremos todos los pares consecutivos del trace
        for i in range(len(trace) - 1):
            current = trace[i]
            next_ = trace[i + 1]

            # Solo nos interesan pares dentro de la misma función
            if current[0] == next_[0]:
                # Si se especificaron funciones objetivo, filtramos por nombre
                if self.target_functions is None or current[0] in self.target_functions:
                    branch_pairs.add((current, next_))

        # Convertimos a lista y ordenamos los pares por número de línea
        return sorted(branch_pairs, key=lambda x: (x[0][1], x[1][1]))


In [97]:
### Obtenemos el coverage de tu implementación con un input de ejemplo:
with BranchCoverage(['categorize_product_quality']) as bcov:
    assert categorize_product_quality(50) == "Good" # test 1
    assert categorize_product_quality(20) == "Poor" # test 2

covered_pairs = bcov.coverage()

# Visualiza tu implementación. Recuerda que debe estar ordenado:
for item in covered_pairs:
    print(item)


(('categorize_product_quality', 3), ('categorize_product_quality', 6))
(('categorize_product_quality', 6), ('categorize_product_quality', 7))
(('categorize_product_quality', 6), ('categorize_product_quality', 8))
(('categorize_product_quality', 8), ('categorize_product_quality', 10))
(('categorize_product_quality', 10), ('categorize_product_quality', 11))
(('categorize_product_quality', 11), ('categorize_product_quality', 3))


## Tests

A continuación puedes testear tu implementación siguiendo el ejemplo que se vio en clases.

In [98]:
def cgi_decode(s: str) -> str:
    """Decode the CGI-encoded string `s`:
       * replace '+' by ' '
       * replace "%xx" by the character with hex number xx.
       Return the decoded string.  Raise `ValueError` for invalid inputs."""

    # Mapping of hex digits to their integer values
    hex_values = {
        '0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
        '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
        'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15,
        'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15,
    }

    t = ""
    i = 0
    while i < len(s):
        c = s[i]
        if c == '+':
            t += ' '
        elif c == '%':
            digit_high, digit_low = s[i + 1], s[i + 2]
            i += 2
            if digit_high in hex_values and digit_low in hex_values:
                v = hex_values[digit_high] * 16 + hex_values[digit_low]
                t += chr(v)
            else:
                raise ValueError("Invalid encoding")
        else:
            t += c
        i += 1
    return t

In [99]:
import unittest

def execute_student_implentation(string_input):
  with BranchCoverage(['cgi_decode']) as bcov:
    cgi_decode(string_input)
  cgi_covered_pairs = bcov.coverage()
  return cgi_covered_pairs

class TestBranchCoverage(unittest.TestCase):
  def test_branch_coverage_implementation_plus(self):
    cgi_covered_pairs = execute_student_implentation("a+b")
    expected_covered_pairs = [
        (('cgi_decode', 8), ('cgi_decode', 9)),
        (('cgi_decode', 8), ('cgi_decode', 10)),
        (('cgi_decode', 8), ('cgi_decode', 11)),
        (('cgi_decode', 8), ('cgi_decode', 12)),
        (('cgi_decode', 8), ('cgi_decode', 15)),
        (('cgi_decode', 9), ('cgi_decode', 8)),
        (('cgi_decode', 10), ('cgi_decode', 8)),
        (('cgi_decode', 11), ('cgi_decode', 8)),
        (('cgi_decode', 12), ('cgi_decode', 8)),
        (('cgi_decode', 15), ('cgi_decode', 16)),
        (('cgi_decode', 16), ('cgi_decode', 17)),
        (('cgi_decode', 17), ('cgi_decode', 18)),
        (('cgi_decode', 17), ('cgi_decode', 32)),
        (('cgi_decode', 18), ('cgi_decode', 19)),
        (('cgi_decode', 19), ('cgi_decode', 20)),
        (('cgi_decode', 19), ('cgi_decode', 21)),
        (('cgi_decode', 20), ('cgi_decode', 31)),
        (('cgi_decode', 21), ('cgi_decode', 30)),
        (('cgi_decode', 30), ('cgi_decode', 31)),
        (('cgi_decode', 31), ('cgi_decode', 17))]
    self.assertEqual (cgi_covered_pairs, expected_covered_pairs,
                      "Test failed: Covered pairs do not match expected values for a+b.")

  def test_branch_coverage_implementation_only_letters(self):
    cgi_covered_pairs = execute_student_implentation("abc")
    expected_covered_pairs = [
        (('cgi_decode', 8), ('cgi_decode', 9)),
        (('cgi_decode', 8), ('cgi_decode', 10)),
        (('cgi_decode', 8), ('cgi_decode', 11)),
        (('cgi_decode', 8), ('cgi_decode', 12)),
        (('cgi_decode', 8), ('cgi_decode', 15)),
        (('cgi_decode', 9), ('cgi_decode', 8)),
        (('cgi_decode', 10), ('cgi_decode', 8)),
        (('cgi_decode', 11), ('cgi_decode', 8)),
        (('cgi_decode', 12), ('cgi_decode', 8)),
        (('cgi_decode', 15), ('cgi_decode', 16)),
        (('cgi_decode', 16), ('cgi_decode', 17)),
        (('cgi_decode', 17), ('cgi_decode', 18)),
        (('cgi_decode', 17), ('cgi_decode', 32)),
        (('cgi_decode', 18), ('cgi_decode', 19)),
        (('cgi_decode', 19), ('cgi_decode', 21)),
        (('cgi_decode', 21), ('cgi_decode', 30)),
        (('cgi_decode', 30), ('cgi_decode', 31)),
        (('cgi_decode', 31), ('cgi_decode', 17))]
    self.assertEqual (cgi_covered_pairs, expected_covered_pairs,
                      "Test failed: Covered pairs do not match expected values for abc.")

  def test_branch_coverage_implementation_empty_string(self):
    cgi_covered_pairs = execute_student_implentation("")
    expected_covered_pairs = [
        (('cgi_decode', 8), ('cgi_decode', 9)),
        (('cgi_decode', 8), ('cgi_decode', 10)),
        (('cgi_decode', 8), ('cgi_decode', 11)),
        (('cgi_decode', 8), ('cgi_decode', 12)),
        (('cgi_decode', 8), ('cgi_decode', 15)),
        (('cgi_decode', 9), ('cgi_decode', 8)),
        (('cgi_decode', 10), ('cgi_decode', 8)),
        (('cgi_decode', 11), ('cgi_decode', 8)),
        (('cgi_decode', 12), ('cgi_decode', 8)),
        (('cgi_decode', 15), ('cgi_decode', 16)),
        (('cgi_decode', 16), ('cgi_decode', 17)),
        (('cgi_decode', 17), ('cgi_decode', 32))]
    self.assertEqual (cgi_covered_pairs, expected_covered_pairs,
                      "Test failed: Covered pairs do not match expected values for empty string.")

  def test_branch_coverage_implementation_full(self):
    cgi_covered_pairs = execute_student_implentation("%20+%40+%21+%40")
    expected_covered_pairs = [
        (('cgi_decode', 8), ('cgi_decode', 9)),
        (('cgi_decode', 8), ('cgi_decode', 10)),
        (('cgi_decode', 8), ('cgi_decode', 11)),
        (('cgi_decode', 8), ('cgi_decode', 12)),
        (('cgi_decode', 8), ('cgi_decode', 15)),
        (('cgi_decode', 9), ('cgi_decode', 8)),
        (('cgi_decode', 10), ('cgi_decode', 8)),
        (('cgi_decode', 11), ('cgi_decode', 8)),
        (('cgi_decode', 12), ('cgi_decode', 8)),
        (('cgi_decode', 15), ('cgi_decode', 16)),
        (('cgi_decode', 16), ('cgi_decode', 17)),
        (('cgi_decode', 17), ('cgi_decode', 18)),
        (('cgi_decode', 17), ('cgi_decode', 32)),
        (('cgi_decode', 18), ('cgi_decode', 19)),
        (('cgi_decode', 19), ('cgi_decode', 20)),
        (('cgi_decode', 19), ('cgi_decode', 21)),
        (('cgi_decode', 20), ('cgi_decode', 31)),
        (('cgi_decode', 21), ('cgi_decode', 22)),
        (('cgi_decode', 22), ('cgi_decode', 23)),
        (('cgi_decode', 23), ('cgi_decode', 24)),
        (('cgi_decode', 24), ('cgi_decode', 25)),
        (('cgi_decode', 25), ('cgi_decode', 26)),
        (('cgi_decode', 26), ('cgi_decode', 31)),
        (('cgi_decode', 31), ('cgi_decode', 17))]
    self.assertEqual (cgi_covered_pairs, expected_covered_pairs, "Test failed: Covered pairs do not match expected values for full string.")

def runTests(testClass):
  loader = unittest.TestLoader()
  suite = loader.loadTestsFromTestCase(testClass)
  runner = unittest.TextTestRunner()
  runner.run(suite)
runTests(TestBranchCoverage)


....
----------------------------------------------------------------------
Ran 4 tests in 0.006s

OK
